As discussed in lectures, being able to sample/generate our own data is a useful skill which allows us to experiment with our algorithms on controlled data. Thus, for this week's lab we will be generating our
own data to work with. 

(a) Lets begin by obtaining the data.

i. Sample 150 x-values from a Normal distribution using a mean of 0 and standard deviation of 10.
Hint: np.random.normal

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

#constants
number_samples = 150
mean = 0
standard_dev = 10

#x values random sample generation
x_samples = np.random.normal(mean, standard_dev, number_samples)

ii. From the x-values construct a design matrix using the features {1,x,x^2}.

In [ ]:
designMatrix_X = np.column_stack((np.ones_like(x_samples), x_samples, x_samples**2))

iii. Sample true values for theta_0, theta_1 and theta_2 using a uniform distribution

In [ ]:
thTrue = np.random.uniform(-1, 1, size=3)

iv. Use your design matrix and the true parameters you obtained to create the y-values for the
regression data. Finally add random noise to the y-values using a Normal distribution with mean
0 and standard deviation of 8.

In [ ]:
meanNoise = 0
standard_dev_noise = 8
yValues = np.dot(designMatrix_X, thTrue) + np.random.normal(meanNoise, standard_dev_noise, size=x_samples.shape[0])

Plot the x-values and their corresponding y-values on a 2D-axis. Your data should look similar
to the data shown in Figure 2a. Hint: pyplot.scatter

In [ ]:
plt.scatter(x_samples, yValues)
plt.axvline(x=0, color='black', linestyle='--')
plt.axhline(y=0, color='black', linestyle='--')
plt.xlabel('x-values')
plt.ylabel('y-values')
plt.title('Generated Data')
plt.show()

vi. Split the data into training, validation and test datasets.

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(designMatrix_X, yValues, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.25, random_state=42)

print(f"Number of training examples: {x_train.shape[0]}")
print(f"Number of validation examples: {x_val.shape[0]}")
print(f"Number of test examples: {x_test.shape[0]}")

(b) Now that we have data we can train our models.

i. Use the Moore-Penrose pseudo-inverse to calculate the closed form solution for the model's parameter values.

ii. How close are the learned parameter values to the true parameter values we used to generate the
data?

In [ ]:
closed = np.dot(np.linalg.pinv(x_train), y_train)
print("Learned Parameters:", closed)
print("True Parameters:", thTrue)

iii. Compute the training error and validation error for the learned regression model.

iv. Create a scatter plot of the individual data points along with the learned regression function,
your plot should look like Figure 2b. Hint: pyplot.plot, this plotting function will give weird
results if the x-values of the data are not sorted. x train[x train[:,1].argsort()] will give you the
design matrix for your training data sorted by the second column (where the x values should be).

In [ ]:
trainingError = np.mean((np.dot(x_train, closed)-y_train)**2)
validationError = np.mean((np.dot(x_val, closed)-y_val)**2)
print("Training Error:", trainingError)
print("Validation Error:", validationError)
sorted = x_train[x_train[:,1].argsort()]
plt.scatter(x_samples, yValues, label="Data Points")
plt.axvline(x=0, color='black', linestyle='--')
plt.axhline(y=0, color='black', linestyle='--')
plt.plot(sorted[:,1], np.dot(sorted, closed), 'r', label="Learned Regression Model")
plt.xlabel('x')
plt.ylabel('y')
plt.title('Regression Model')
plt.legend()
plt.show()

v. Repeat the above process using Gradient Descent to train your model. In addition, plot the
training error of your regression model over time (observe or capture the training error every 20
parameter updates/time steps). Your plot should look like Figure 2c.

In [ ]:
def gradientDescent(matrixX, yTrained, theta, learnRate, numberIterations):
    thetaI = theta
    m = matrixX.shape[0]
    trainErrors = []
    for i in range(numberIterations):
        thetaI = thetaI - (learnRate * ((np.dot(matrixX.T, (np.dot(matrixX, thetaI)) - yTrained)))/m)
        trainError = np.mean(((np.dot(matrixX, thetaI)) - yTrained)**2)
        trainErrors.append(trainError)
        if i % 20 == 0:
            print(f"Iteration {i}: Training Error = {trainError}")
    return thetaI, trainErrors

thetaI = np.zeros(3)
learnRate = 0.000001
numberIterations = 100
thetaGradientDescent, trainErrors = gradientDescent(x_train, y_train, thetaI, learnRate, numberIterations)

plt.plot(np.arange(len(trainErrors)), trainErrors)
plt.axvline(x=0, color='black', linestyle='--')
plt.axhline(y=0, color='black', linestyle='--')
plt.xlabel('Time Step')
plt.ylabel('Training Error')
plt.title('Error over Time')
plt.show()


(c) We will now experiment with overfitting and regularization.

i. Begin by appending a third feature to your design matrix for x^3.

In [ ]:
xTrain_3 = np.hstack((np.ones((x_train.shape[0], 1)), x_train, np.square(x_train[:, 1]).reshape(-1, 1)))

ii. Train a model using Gradient Descent with the new design matrix. Repeat the process used above
in Question 4b. Note, we are now using a third-order polynomial to fit data which was generated
using a second-order polynomial. Our function is, thus, more complicated than is necessary to
fit the data and as a result will overfit.

In [ ]:
thetaI = np.random.uniform(low=-1, high=1, size=xTrain_3.shape[1])
learnRate = 0.000001
numberIterations = 100
thetaGradientDescent_3, trainErrors_3 = gradientDescent(xTrain_3, y_train, thetaI, learnRate, numberIterations)

iii. Repeat the training process one final time, this time use regularization when training the third-
order polynomial model.

iv. Compare your results of the three gradient descent based models, which model achives the best
final training error? Which model trains the fastest? Which model achieves the best validation
error? Can you see any visible difference in the function approximation (fit of the data) by the
models or in the learned parameter values? Did any of these models achieve a lower training or
validation error than the closed form solution?

In [ ]:
def gradientDescentWithRegularization(matrixX, yTrained, theta, learnRate, numberIterations, lambdaReg):
    thetaI = theta
    m = matrixX.shape[0]
    trainErrors = []
    for i in range(numberIterations):
        thetaI = thetaI - (learnRate * ((np.dot(matrixX.T, (np.dot(matrixX, thetaI)) - yTrained)) + lambdaReg * thetaI)/m)
        trainError = np.mean(((np.dot(matrixX, thetaI)) - yTrained)**2)
        trainErrors.append(trainError)
        if i % 20 == 0:
            print(f"Iteration {i}: Training Error = {trainError}")
    return thetaI, trainErrors
thetaI = np.random.uniform(low=-1, high=1, size=xTrain_3.shape[1])
learnRate = 0.000001
numberIterations = 100
lambdaReg = 0.001
thetaGradientDescentReg, trainErrorsReg = gradientDescentWithRegularization(xTrain_3, y_train, thetaI, learnRate, numberIterations, lambdaReg)

plt.plot(np.arange(len(trainErrors)), trainErrors, label='Second-order polynomial')
plt.plot(np.arange(len(trainErrors_3)), trainErrors_3, label='Third-order polynomial')
plt.plot(np.arange(len(trainErrorsReg)), trainErrorsReg, label='Third-order polynomial with Regularization')
plt.axvline(x=0, color='black', linestyle='--')
plt.axhline(y=0, color='black', linestyle='--')
plt.xlabel('Time Step')
plt.ylabel('Training Error')
plt.title('Error over Time')
plt.legend()
plt.show()
print(f"Second-order polynomial training error: {trainErrors[-1]}")
print(f"Third-order polynomial training error: {trainErrors_3[-1]}")
print(f"Third-order polynomial with regularization training error: {trainErrorsReg[-1]}")

x_plot = np.linspace(-5, 5, 100)
y_plot = thetaGradientDescent[0] + thetaGradientDescent[1]*x_plot + thetaGradientDescent[2]*x_plot**2
plt.plot(x_plot, y_plot, label='Second-order polynomial')
y_plot_3 = thetaGradientDescent_3[0] + thetaGradientDescent_3[1]*x_plot + thetaGradientDescent_3[2]*x_plot**2 + thetaGradientDescent_3[3]*x_plot**3
plt.plot(x_plot, y_plot_3, label='Third-order polynomial')
y_plot_reg = thetaGradientDescentReg[0] + thetaGradientDescentReg[1]*x_plot + thetaGradientDescentReg[2]*x_plot**2 + thetaGradientDescentReg[3]*x_plot**3
plt.plot(x_plot, y_plot_reg, label='Third-order polynomial with regularization')

# sort_idx = x_test.argsort(axis=0)
# x_test_sorted = x_test[sort_idx]
# y_test_sorted = y_test[sort_idx]
# plt.scatter(x_test_sorted, y_test_sorted, label='Test data')
# plt.xlabel('x')
# plt.ylabel('y')
# plt.title('Fitted function')
# plt.legend()
# plt.show()


Compute the final test error for all four of your models. Which model obtains the lowest test
error? Did the regularisation improve the test error performance of the third-order polynomial
model? Which of the three gradient descent models achieved the lowest test error? Would you
say it is better to use higher-order polynomials and regularize or use a model which only uses the
necessary features to fit the data.

In [ ]:
testError = np.mean((np.dot(x_test, closed)-y_test)**2)
print("Test Error (Second Order Polynomail):", testError)
xTest_3 = np.hstack((np.ones((x_test.shape[0], 1)), x_test, np.square(x_test[:, 1]).reshape(-1, 1)))
testError_3 = np.mean((np.dot(xTest_3, thetaGradientDescent_3)-y_test)**2)
print("Test Error (Third Error Polynomial):", testError_3)
testErrorReg = np.mean((np.dot(xTest_3, thetaGradientDescentReg)-y_test)**2)
print("Test Error (Third Error Polynomial with regularization):", testErrorReg)

# # Fit the first-order polynomial model
# model1.fit(x_train, y_train)
# y_pred1 = model1.predict(x_test)
# mse1 = mean_squared_error(y_test, y_pred1)

# # Fit the second-order polynomial model
# model2.fit(x_train, y_train)
# y_pred2 = model2.predict(x_test)
# mse2 = mean_squared_error(y_test, y_pred2)

# # Fit the third-order polynomial model
# model3.fit(x_train, y_train)
# y_pred3 = model3.predict(x_test)
# mse3 = mean_squared_error(y_test, y_pred3)

# # Fit the third-order polynomial model with regularization
# model4.fit(x_train, y_train)
# y_pred4 = model4.predict(x_test)
# mse4 = mean_squared_error(y_test, y_pred4)
